 # JuliaHPC @ CERMICS
 
 ## Block 2: Parallelisation

 ### Session 2: Threads

Cyrille Vessaire

2021/02/10

In [51]:
#import Pkg
#Pkg.add([
#         "BenchmarkTools",
#         "LightGraphs",
#         "GraphPlot",
#         "LinearAlgebra",
#         "Profile",
#         "ProfileSVG",
#         "ProgressMeter",
#         "Random",
#         "Traceur",
#         "ThreadTools"
#])

# A. Parallelisation

## A.1) What is Parallelisation

Parallelisation is an idea as old as computer science itself. 


The basic idea comes from one question:

when one has a problem that take 10 hours to solve on one machine, 

wouldn't it be possible to solve it in 5 hours with two machines?

Parallelism is thus an old idea on how to make <span style="color:red">computation faster </span>.

## A.2) Level of parallelism

There are three main level for parallelisation
- Bit-level parallelism
- Instruction level parallelism
- Task level parallelism

The first two are made at the processor level.
We will only tackle <span style="color:red">Task level parallelism </span>.

## A.3) Parallelism and computer hardware

Depending on your hardware, you won't do parallelisation with the same tools.
Here is an illustration of the common part of a computer:

 <center>
    <img src="./tikzgraphics/tikz_computers.png" >
 </center>

In this presentation on multi-threading, we will only consider parallelism on <span style="color:red">one machine</span>, and with the <span style="color:red">CPU</span>.

# B. Multi-threading

## B.1) Threads vs Processes

A process is an instance of a computer program.
In modern OS, due to security issue, processes don't share the same memory.

Meanwhile, threads are <span style="color:red">part</span> of <span style="color:red">one process</span>, and have <span style="color:red">direct access to the same data</span>.

Pros and Cons of Threads

Pros:
1. <span style="color:red">Threads share the same memory register</span>. They can use the same variables, array, etc.
2. They are the <span style="color:red">easiest way</span>. to implement parallelisation*

Cons:
1. <span style="color:red">Threads share the same memory register</span>.. They can modify each others data and output false results
2. You can't share threads on <span style="color:red">multiple computers</span>.. Parallelisation through multi-threading can work on only one process.

*When you have a functionning serial code


## B.2) Threading in Julia

### a) Preparation

First, one can change environment variable so that julia can use multiple threads.

In windows, write
```shell
foo@bar:~$ set JULIA_NUM_THREADS=4
```

In linux, do
```shell
foo@bar:~$ export JULIA_NUM_THREADS=4
```

You can the number of available threads with

In [11]:
using Base.Threads
nthreads()

4

One can also set the number of threads by using the -t argument when launching REPL (on julia 1.5).

```shell
foo@bar:~$ julia -t 4
```

You can also check that threading work well with tests such as

In [12]:
@threads for i in 1:10
    println(Threads.threadid())
end

1
3
3
4
4
2
2
2
1
1


To use multi-threading in notebooks, you need to <span style="color:red">create a kernel with multiple threads</span>.
To do so, write in REPL:

In [ ]:
using IJulia
IJulia.installkernel("Julia 4 Threads", env=Dict(
    "JULIA_NUM_THREADS" => "4",
))

#### b) Using Multi-Threading in Julia

To do some threadings on `for` loop, use the `@threads` macro



In [3]:
for i=1:10
    println(Threads.threadid())
end

1
1
1
1
1
1
1
1
1
1


In [4]:
using Base.Threads
@threads for i=1:10
    println(threadid())
end

1
4
3
3
4
2
1
1
2
2


However, one must be careful, as there can be error induced by doing parallel computing

In [14]:
n = 10^7
a = ones(n)
@time for i=1:(n-1)
    a[i]+= a[i+1]
end

n_error = 0
for i=1:(n-1)
    if a[i]!=2
        n_error += 1
    end
end
println(n_error)

  2.765876 seconds (70.00 M allocations: 1.192 GiB, 13.25% gc time)
0


In [3]:
using Base.Threads

n = 10^7
a = ones(n)
@time @threads for i=1:(n-1)
    a[i]+= a[i+1]
end

n_error = 0
for i=1:(n-1)
    if a[i]!=2
        n_error += 1
    end
end
println(n_error)

  0.662718 seconds (60.02 M allocations: 916.470 MiB, 24.39% gc time)
3


We need to be careful of race competition situation.

A race competition happens when two threads want to access and modify a given memory cell.

In [39]:
using Base.Threads

n = 10^7
a = ones(n)
for i=1:(n-1)
    a[i]+= a[i+1]
end

b = ones(n)
@threads for i=1:(n-1)
    b[i]+= b[i+1]
end

In [36]:
n_error = 0
for i=1:n
    if a[i]!=b[i]
        n_error += 1
    end
end
println(n_error)

3


In [37]:
error_index = zeros(n_error)
n_error_bis = 0
for i=1:n
    if a[i]!=b[i]
        n_error_bis += 1
        if n_error < 10
            error_index[n_error_bis] = i
        end
    end
end
println(error_index)

[2.5e7, 5.0e7, 7.5e7]


### c) Preventing race conditions

`Threads.Atomic` can be used to prevent race conditions

In [8]:
i = Threads.Atomic{Int}(0);

ids = zeros(4);

old_is = zeros(4);

@threads for id in 1:4
    old_is[id] = Threads.atomic_add!(i, id)
    ids[id] = id
end

In [9]:
old_is

4-element Array{Float64,1}:
 0.0
 4.0
 1.0
 6.0

In [10]:
ids

4-element Array{Float64,1}:
 1.0
 2.0
 3.0
 4.0

In [11]:
i

Atomic{Int64}(10)

Another example

In [25]:
n = 10^8
acc_1 = Ref(0)
@time @threads for i in 1:n
    acc_1[] += 1
end

acc_2 = Atomic{Int64}(0)
@time @threads for i in 1:n
    atomic_add!(acc_2, 1)
end

println(acc_1[])
println(acc_2[])

  3.965083 seconds (200.02 M allocations: 2.981 GiB, 18.03% gc time)
  1.922673 seconds (19.84 k allocations: 1.027 MiB)
27891725
100000000


We can also prevent race conditions by using <span style="color:red">locks</span>

In [10]:
using ThreadTools

l = SpinLock()
a = [0]
@threads for i = 1:10000
    lock(l) do
        a[] += 1
    end
end

b = [0]
@threads for i = 1:10000
    b[] += 1
end

println(a[])
println(b[])

10000
3633


### d) Spawn and synchronization

You may want to use threads to do multiple computation, and use `@spawn` to do so.

However, while `@threads` is synchronius, `@spawn` isn't.

You may need to <span style="color:red">synchronize spawned threads</span>.

In [ ]:
function withthreads()
    arr = zeros(Int, 10)
    Threads.@threads for i in 1:10
       sleep(3 * rand())
       arr[i] = i
    end
    println("with @threads: $arr")
end


function withspawn()
    arr = zeros(Int, 10)
    for i in 1:10
        Threads.@spawn begin
            sleep(3 * rand())
            arr[i] = i 
        end
    end
    println("with @spawn: $arr")
end

In [30]:
       

function withsync()
    arr = zeros(Int, 10)
    @sync begin
        for i in 1:10
           Threads.@spawn begin
               sleep(3 * rand())
               arr[i] = i
           end
        end
    end
    println("with @sync: $arr")
end

withthreads()
withspawn()
withsync()

with @threads: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
with @spawn: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
with @sync: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


What exactly happened?

In [9]:
using Base.Threads

function withspawn_bis()
    arr = zeros(Int, 10)
    for i in 1:10
        println(i)
        Threads.@spawn begin
            println("Spawn i : ", $i, ", thread : ", $threadid())
            sleep(3 * rand())
            println("Spawn ", $i, " woke up")
            arr[i] = i
        end
    end
    # sleep(4)
    println("with @spawn: $arr")
end
withspawn_bis()

1
2
3
4
5
Spawn i : 1, thread : 3
6
7
Spawn i : 6, thread : 3
8
Spawn i : 7, thread : 3
9
10
Spawn i : 8, thread : 3
with @spawn: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Spawn i : 9, thread : 3
Spawn i : 10, thread : 3
Spawn i : 3, thread : 4
Spawn i : 4, thread : 1
Spawn i : 5, thread : 1
Spawn i : 2, thread : 2
Spawn 4 woke up
Spawn 9 woke up
Spawn 7 woke up
Spawn 1 woke up
Spawn 5 woke up
Spawn 10 woke up
Spawn 6 woke up
Spawn 2 woke up
Spawn 3 woke up
Spawn 8 woke up


Can you get good performance with `@spawn`?

Let us take the example of a <span style="color:red">reccursive function</span>.

In [14]:
function Fibo(n)
if n < 2
    return n
    else
        return Fibo(n-1)+Fibo(n-2)
    end
end
@time Fibo(30)

  0.005021 seconds


832040

In [15]:
import Base.Threads.@spawn

function fib(n::Int)
    if n < 2
        return n
    end
    t = @spawn fib(n - 2)
    return fib(n - 1) + fetch(t)
end

@time fib(30)

  1.336152 seconds (8.18 M allocations: 1.137 GiB, 45.76% gc time)


832040

Using threads can create a lot of overhead.
We can get faster by reducing the time needed by only using threads when the sequential code is slower 

In [36]:
function fib_bis(n::Int)
    if n < 2
        return n
    elseif n > 25
        t = @spawn fib_bis(n - 2)
        return fib_bis(n - 1) + fetch(t)
    else
        return fib_bis(n - 1) + fib_bis(n-2)
    end
end

@time fib_bis(30)

  0.016490 seconds (3.56 k allocations: 101.931 KiB)


832040

In [24]:
a = @time Fibo(50)
b = @time fib_bis(50)
a == b

 68.588469 seconds
157.145740 seconds (52.41 M allocations: 2.700 GiB, 0.79% gc time)


true

We can go even faster by calling the sequential function itself

In [28]:
function fib_tri(n::Int)
    if n <= 25
        return Fibo(n)
    else
        t = @spawn fib_tri(n - 2)
        return fib_tri(n - 1) + fetch(t)
    end
end

fib_tri (generic function with 1 method)

In [37]:
a = @time Fibo(45)
b = @time fib_bis(45)
c = @time fib_tri(45)
a == b == c

  6.079204 seconds
 12.620620 seconds (3.63 M allocations: 66.498 MiB, 0.92% gc time)
  1.815553 seconds (142.76 k allocations: 13.305 MiB)


true

As with the previous `@threads` macro, you still need to be careful about race conditions

In [ ]:
using Base.Threads

n = 10^6
a = ones(n)
for i=1:(n-1)
    a[i]+= a[i+1]
end

b = ones(n)
for i=1:(n-1)
    Threads.@spawn begin
        b[i]+= b[i+1]
    end
end 

In [43]:
n_error = 0
for i=1:n
    if a[i]!=b[i]
        n_error += 1
    end
end
println(n_error)

1222


### e) Composable Threading

In julia, you can <span style="color:red">compose threading</span>.

This means that if you use package that already use multi-threading, they will still use it if you also want to use multithreading.

Example of package using multi-threading

In [2]:
using LinearAlgebra, Base.Threads

n = 5000
A = randn(n,n)
B = randn(n,n)
C = zeros(n,n)
for i=1:4
    @time C = A*B
end

D = zeros(n,n)
@threads for i=1:4
    @time D = A*B
end

  3.392750 seconds (2.70 M allocations: 320.502 MiB, 3.69% gc time)
  2.654315 seconds (2 allocations: 190.735 MiB)
  2.871894 seconds (2 allocations: 190.735 MiB, 0.68% gc time)
  2.887381 seconds (2 allocations: 190.735 MiB)
  4.051232 seconds (47 allocations: 762.943 MiB)
  7.996629 seconds (535 allocations: 572.233 MiB)
 11.461857 seconds (936 allocations: 381.515 MiB)
 14.672010 seconds (1.31 k allocations: 190.797 MiB)
